<div style="border:solid green 2px; padding: 20px">
<b>Кирилл, привет!</b>

Меня зовут Евгений Дащенко, и я буду проверять твой проект. Предлагаю общаться на «ты» 😊 Но если это не удобно - дай знать, и мы перейдем на "вы". 

Моя основная цель — не указать на совершенные тобою ошибки, а поделиться своим опытом и помочь тебе стать аналитиком данных. Ты уже проделал большую работу над проектом, но давай сделаем его еще лучше. Ниже ты найдешь мои комментарии - пожалуйста, не перемещай, не изменяй и не удаляй их. Я буду использовать цветовую разметку:

<div class="alert alert-danger">
<b>Комментарий ревьюера:</b> Так выделены самые важные замечания. Без их отработки проект не будет принят. </div>

<div class="alert alert-warning">
<b>Комментарий ревьюера:</b> Так выделены небольшие замечания. Одна-два таких замечания в проекте допустимы, но если их будет больше - тебе будет необходимо внести исправления. Это как тестовое задание при приеме на работу: очень много мелких ошибок могут стать причиной отказа кандидату. 

</div>

<div class="alert alert-success">
<b>Комментарий ревьюера:</b> Так я выделяю все остальные комментарии.</div>

Давай работать над проектом в диалоге: если ты что-то меняешь в проекте или отвечаешь на мои комменатри — пиши об этом. Мне будет легче отследить изменения, если ты выделишь свои комментарии:
<div class="alert alert-info"> <b>Комментарий студента:</b> Например, вот так.</div>

Всё это поможет выполнить повторную проверку твоего проекта оперативнее. 

# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Здорово, когда есть вступление в сам проект, каждый сможет быстрее разобраться в концепции / деталях работы. 

</div>

## Шаг 1. Откройте файл с данными и изучите общую информацию


Импортируем необходимые библиотеки:

In [1]:
import pandas as pd 
data = pd.read_csv('/datasets/data.csv')
from pymystem3 import Mystem
m = Mystem()

Прочитаем исходный файл и выведем первые 10 строк:

In [2]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Дополнительно посмотрим общую информацию о таблице:

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Рассмотрим полученную информацию подробнее:
1. Все столбцы имеют "хорошие" названия: в нижнем регистре, с подчеркиванием, без лишних пробелов. Замена названий не требуется.
1. Количество значений в столбцах различаются. Это говорит о том, что в данных есть пустые значения. А именно в колонках `'days_employed'` и `'total_income'` по 2174 пустых значения.  
1. Данные о трудовом стаже. Стаж должен быть в днях, но имеет тип `float`. Среди значений есть отрицательные. Так же среди значений встречаются значения не характерные для трудового стажа, например, 340266.072047, а это более 950 лет. Возможно, была применена некая функция при расчете трудового стажа.
1. Среди данных об образовании клиента встречаются значения с разными регистрами.
1. Данные о ежемесячном доходе имеют тип `float`.

**Вывод**

В предоставленной таблице встречаются проблемные данные и пустые значения. Исследуем их более подробно.

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Достаточно детально изучили наши первичные данные, давай теперь их обработаем. 
</div>

## Шаг 2. Предобработка данных

### Обработка пропусков

По разнице значений в информации мы сделали вывод, что пропуски в столбцах `'days_employed'` и `'total_income'` есть. Находим пропуски в столбце `'total_income'` методом `isna()` и выводим их на экран

In [4]:
data[data['total_income'].isnull()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


In [5]:
data[data['total_income'].isna()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


По полученной таблице можно предположить, что если в `'days_employed'` значение `NaN`, то и в `'total_income'` тоже будет `NaN`

Проверим это условием:

In [6]:
data.loc[(data['total_income'].isna())&(data['days_employed'].isna())].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Посчитаем количество таких строк

In [7]:
len(data.loc[(data['total_income'].isna())&(data['days_employed'].isna())])

2174

Таким образом, мы получили, что 2174 строки одновременно содержат `NaN` в столбцах `'days_employed'` и `'total_income'`. Возможных причин для пустых значений может быть несколько: клиент не стал указывать трудовой стаж и доход, отсутствует ежемесячный доход и стаж работы, либо вследствие технического сбоя. 
Удалять строки с пустыми данными нельзя, так как они содержат важные для нас данные по количеству детей и семейному положению. Пока что будем считать, что у таких клиентов отсутствует трудовой стаж и ежемесячный доход. Произведем замену `NaN` на 0 методом `fillna()`

In [8]:
data['total_income'] = data['total_income'].fillna(0)
data['days_employed'] = data['days_employed'].fillna(0)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

В том числе метод `fillna` и `astype` можно применять к нескольким столбцам одновременно.
    
Если столбцов более 2–ух, то рекомендую применять цикл `for`.
    
Пример: 
    
`df.astype({ "col1": "float64", "col2": "int64"})`
    
</div>

```python

columns = [col1, col2, col3]

for i in columns:
    df[i] = df[i].fillna(0)

```

Проверяем, что во всех столбцах есть значения.

**Вывод**

Выявили пропущенные значения в ежемесячном доходе и стаже работы. Установили, что в строках, где нет дохода, так же отсутствует стаж работы. Для таких записей сделали замену пропусков на 0, посчитав, что у клиента ежемесячный доход отсутствует.

### Замена типа данных

В общей информации о таблице выявили столбцы с типом `float`: `'days_employed'` и `'total_income'`.

Переведем ежемесячный доход и стаж работы в целые числа, так как стаж считается в днях, а ежемесячный доход будем считать с точностью до целых. Для перевода воспользуемся `astype()`:

In [9]:
data['total_income'] = data['total_income'].astype('int')

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


In [11]:
data['days_employed'] = data['days_employed'].astype('int')

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Произвели замену типов`float` на `int` для столбцов `'days_employed'` и `'total_income'`. 

### Обработка дубликатов

Перед тем, как приступать к поиску явных дубликатов, исследуем каждый столбец на артефакты/уникальные значения

#### Проверка столбца `'children'`

Посмотрим на уникальные значения столбца `'children'` используем метод `unique()`

In [13]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Выдающиеся значения '-1' и '20'. Посмотрим сколько записей по каждому значению методом `value_counts()`

In [14]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Исходя из этих данных, можно заметить резкое уменьшение числа записей с ростом количества детей. Для 5 детей всего 9 записей. Вероятнее всего, значение '20' появилось вследствие опечатки. Поэтому отнесем такие записи к записям, где двое детей. Записи со значение '-1' отнесем к записям с '1'. Возможно, стоит добавить валидацию на значения поля на этапе формирования заявки на кредит. Воспользуемся методом `replace()`

In [15]:
data['children'] = data['children'].replace(20, 2)

In [16]:
data['children'] = data['children'].replace(-1, 1)

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

А если бы мы посмотрели на долю этих значений, то смогли бы принять решение в сторону избавления от этих данных ? 
    
</div>

In [17]:
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Столбец с количеством детей привели в порядок.

#### Проверка столбца `'income_type'`

Проверяем на уникальность `income_type`

In [18]:
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

С типами клиентов всё в порядке.

#### Проверка столбца `'dob_years'`

Посмотрим на уникальные значения в `'dob_years'`

In [19]:
data['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

Среди данных о возрасте есть значение `0`. Для исключения нулевых данных возраста, возможно стоит предусмотреть обязательное заполнение данного поля клиентом, с добавлением валидации на ненулевое значение.

In [20]:
#Находим количество таких записей в таблице
len(data.loc[data['dob_years'] == 0 ])

101

101 запись в таблице с указанным возрастом `0`. Для замены нулевого значения поступим следующим образом. Для каждой записи с нулевым возрастом посмотрим, какой был указан тип клиента. Для каждого типа клиента рассчитаем среднее значение возраста. А потом заменим нулевое значение на среднее, в зависимости от его типа. Среднее значение было использовано, так как шкала возраста более менее непрерывна.

In [21]:
#Создаем новую таблицу c нулевым возрастом и типом клиента, применив фильтрацию к исходной
data_years_type = data[['dob_years','income_type']].loc[data['dob_years'] == 0 ]
#находим все типы клиента с возрастом ноль
data_years_type['income_type'].value_counts()

сотрудник      55
пенсионер      20
компаньон      20
госслужащий     6
Name: income_type, dtype: int64

Установили, что для следующих типов клиента в записях есть нулевые значения возраста:
1. сотрудник      
1. пенсионер
1. компаньон
1. госслужащий

Для каждого найденного типа рассчитаем средний возраст. Потом подставим его в нулевые значения:

In [22]:
#Находим среднее значение для типа 'сотрудник' и сразу переводим в 'int'
mean_1 = data['dob_years'].loc[data['income_type'] == 'сотрудник'].mean().astype('int')

#Находим среднее значение для типа 'пенсионер' и сразу переводим в 'int'
mean_2 = data['dob_years'].loc[data['income_type'] == 'пенсионер'].mean().astype('int')

#Находим среднее значение для типа 'компаньон' и сразу переводим в 'int'
mean_3 = data['dob_years'].loc[data['income_type'] == 'компаньон'].mean().astype('int')

#Находим среднее значение для типа 'компаньон' и сразу переводим в 'int'
mean_4 = data['dob_years'].loc[data['income_type'] == 'госслужащий'].mean().astype('int')



In [23]:
#Присваиваем нулевым значениям рассчитанные средние значения возраста по каждой категории
data.loc[(data['income_type'] == 'сотрудник')&(data['dob_years'] == 0), 'dob_years'] = mean_1
data.loc[(data['income_type'] == 'пенсионер')&(data['dob_years'] == 0), 'dob_years'] = mean_2
data.loc[(data['income_type'] == 'компаньон')&(data['dob_years'] == 0), 'dob_years'] = mean_3
data.loc[(data['income_type'] == 'госслужащий')&(data['dob_years'] == 0), 'dob_years'] = mean_4

In [24]:
#Проверяем, что в таблице не осталось нулевых значений возраста
len(data.loc[data['dob_years'] == 0 ])

0

Столбец возраста привели в порядок. Все нулевые значения возраста заменили на средние значения в зависимости от типа клиента

#### Проверка столбца `'education'`

При первичном осмотре таблицы в глаза бросались значения в `'education'` с разными регистрами. Посмотрим на все значения:

In [25]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Приведем все значения в `'education'` к нижнему регистру, по аналогии с `family_status`

In [26]:
data['education'] = data['education'].str.lower()
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Значения в `'education'` привели в порядок

#### Проверка столбца `'education_id'`

Проверяем значения в `'education_id'`

In [27]:
data['education_id'].unique()

array([0, 1, 2, 3, 4])

Артефактов не обнаружено, столбец в порядке

#### Проверка столбца `'family_status'`

Посмотрим на уникальные значения столбца `'family_status'`

In [28]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Поправим значение 'Не женат/ не замежуем', чтобы все значения были в нижнем регистре. Воспользуемся методом `str.lower()`

In [29]:
data['family_status'] = data['family_status'].str.lower()
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

Семейные статусы приведены в порядок

#### Проверка столбца `'family_status_id'`

Теперь посмотрим на уникальные значения столбца `'family_status_id'`

In [30]:
data['family_status_id'].unique()

array([0, 1, 2, 3, 4])

Артефактов не обнаружено

#### Проверка столбца `'debt'`

Проверим уникальные значения в `'debt'`

In [31]:
data['debt'].unique()

array([0, 1])

Артефактов не обнаружено

#### Проверка столбца `'gender '`

In [32]:
data['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

Среди значений пола есть значение 'XNA'. Проверим количество записей с таким значеним

In [33]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

В данных только одна запись с пропущенным полом клиента. Удалим её из выборки. Возможно, при обработке данных стоит учесть валидацию на ввод поля "Пол".

In [34]:
#Присвоим data отфильтрованную версию себя, без значения 'XNA'
data = data.loc[data['gender'] != 'XNA']

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Отлично, избавились от артефактов в наших данных 👍
    
Можно было бы в том числе посмотреть долю значений, которую мы потеряли, чтобы убедиться что их немного.
    
</div>

In [35]:
#Проверим, что в data в столбце 'gender' отсутствует пропущенное значение
data['gender'].unique()

array(['F', 'M'], dtype=object)

Столбец `'gender'` привели в порядок

#### Проверка столбца `'total_income '`

В столбце `'total_income'` ранее мы меняли пустые значения на `'0'`.

In [36]:
data['total_income'].value_counts()

0         2174
102816       3
126262       3
144533       3
107832       3
          ... 
109583       1
101387       1
138249       1
280240       1
229304       1
Name: total_income, Length: 18606, dtype: int64

Теперь заполним нулевые значения на медианные для каждой категории `'income_type '`. То есть, сделаем по аналогии столбца `'dob_years'`

In [37]:
#Создаем новую таблицу c нулевым доходом и типом клиента, применив фильтрацию к исходной
data_income_type = data[['income_type','total_income']].loc[data['total_income'] == 0 ]
#находим все типы клиента с нулевым доходом
data_income_type['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Таким образом получили, что в пяти типах присутствуют нулевые доходы. Посчитаем медианные значения для этих 5 типов:

In [38]:
#Медианное значение для типа 'сотрудник'
median_1 = data['total_income'].loc[data['income_type'] == 'сотрудник'].median().astype('int')

#Медианное значение для типа 'компаньон'
median_2 = data['total_income'].loc[data['income_type'] == 'компаньон'].median().astype('int')

#Медианное значение для типа 'пенсионер'
median_3 = data['total_income'].loc[data['income_type'] == 'пенсионер'].median().astype('int')

#Медианное значение для типа 'госслужащий'
median_4 = data['total_income'].loc[data['income_type'] == 'госслужащий'].median().astype('int')

#Медианное значение для типа 'предприниматель'
median_5 = data['total_income'].loc[data['income_type'] == 'предприниматель'].median().astype('int')

Теперь заполним нулевые значения полученными медианными значениями для каждой категории:

In [39]:
data.loc[(data['income_type'] == 'сотрудник')&(data['total_income'] == 0), 'total_income'] = median_1
data.loc[(data['income_type'] == 'компаньон')&(data['total_income'] == 0), 'total_income'] = median_2
data.loc[(data['income_type'] == 'пенсионер')&(data['total_income'] == 0), 'total_income'] = median_3
data.loc[(data['income_type'] == 'госслужащий')&(data['total_income'] == 0), 'total_income'] = median_4
data.loc[(data['income_type'] == 'предприниматель')&(data['total_income'] == 0), 'total_income'] = median_5

Проверим, что нулевых значений в `'total_income'` не осталось

In [40]:
data['total_income'].value_counts()

133546    1106
162379     508
110179     414
139034     148
107832       3
          ... 
101387       1
138249       1
280240       1
390148       1
264193       1
Name: total_income, Length: 18607, dtype: int64

Столбец `'total_income'` привели в порядок

#### Проверка столбца `'days_employed '`

Проверяем столбец `'days_employed '` методом `value_counts()`

In [41]:
data['days_employed'].value_counts()

 0         2174
-133         16
-327         16
-438         15
-223         14
           ... 
-11220        1
-7126         1
 373727       1
 355300       1
-4058         1
Name: days_employed, Length: 9087, dtype: int64

Самый проблемный столбец в данных. Тут и отрицательные значения, и пропуски (заменили на нулевые значения), и не характерные для трудового стажа количество дней. Отрицательные значения могли появиться здесь из-за дополнительного символа '-', который использовали как "тире".

Попробуем привести его в порядок: заменим отрицательные значения

In [42]:
#Применяем к значениям столбца 'days_employed' функцию вычисления модуля числа abs()
data['days_employed'] = abs(data['days_employed'])
#Проверяем, что отрицательных значений больше нет
data['days_employed'].value_counts()

0         2174
133         16
327         16
438         15
223         14
          ... 
350691       1
329638       1
346609       1
360739       1
6219         1
Name: days_employed, Length: 9087, dtype: int64

Отрицательные значения перевели в соответствующие положительные

Некоторые значения слишком огромные. Например, 360739. Если предположить, что данное число не в днях, а в часах, тогда при переводе в года это будет ~40 лет. Для категории "Пенсионер" такой стаж укладывается в здравый смысл. Возможно, данные собирались в общую таблицу из разных источников. Поэтому для категории пенсионеров все значения трудового стажа схожи и одинакого нехарактерны.

In [43]:
#Посмотрим на минимальное значение стажа для категории 'пенсионер'
data['days_employed'].loc[(data['income_type'] == 'пенсионер')&(data['days_employed'] != 0)].min()

328728

Минимальный стаж категории 'пенсионер' при переводе в года получился 37,5 лет. То есть, вероятно, значения для пенсионеров не в днях, а в часах. Поделим все значения стажа на 24 для 'пенсионеров'

In [44]:
data.loc[(data['income_type'] == 'пенсионер'), 'days_employed'] = (data['days_employed'] / 24).astype('int')

In [45]:
data['days_employed'].loc[data['income_type'] == 'пенсионер'].value_counts()

0        413
15572      7
14542      5
14621      5
14159      5
        ... 
15628      1
16171      1
14124      1
14128      1
16405      1
Name: days_employed, Length: 2096, dtype: int64

Заменим нулевые значения на медианные для каждой категории, в которой они есть (по аналогии со столбцом `'income_income'`). Хотя в данном случае, можно было бы воспользоваться и средним, так как каких-либо выдающихся значений стажа в категориях нет.

Ищем, для каких категорий клиентов есть пустые значения в стаже:

In [46]:
#Создаем новую таблицу c нулевыми доходом и типом клиента, применив фильтрацию к исходной
data_days_type = data[['income_type','days_employed']].loc[data['days_employed'] == 0 ]
#находим все типы клиента с нулевым доходом
data_days_type['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

In [47]:
#Медианное значение для типа 'сотрудник'
median_days_1 = data['days_employed'].loc[data['income_type'] == 'сотрудник'].median().astype('int')

#Медианное значение для типа 'компаньон'
median_days_2 = data['days_employed'].loc[data['income_type'] == 'компаньон'].median().astype('int')

#Медианное значение для типа 'пенсионер'
median_days_3 = data['days_employed'].loc[data['income_type'] == 'пенсионер'].median().astype('int')

#Медианное значение для типа 'госслужащий'
median_days_4 = data['days_employed'].loc[data['income_type'] == 'госслужащий'].median().astype('int')

#Медианное значение для типа 'предприниматель'
median_days_5 = data['days_employed'].loc[data['income_type'] == 'предприниматель'].median().astype('int')

In [48]:
data.loc[(data['income_type'] == 'сотрудник')&(data['days_employed'] == 0), 'days_employed'] = median_days_1
data.loc[(data['income_type'] == 'компаньон')&(data['days_employed'] == 0), 'days_employed'] = median_days_2
data.loc[(data['income_type'] == 'пенсионер')&(data['days_employed'] == 0), 'days_employed'] = median_days_3
data.loc[(data['income_type'] == 'госслужащий')&(data['days_employed'] == 0), 'days_employed'] = median_days_4
data.loc[(data['income_type'] == 'предприниматель')&(data['days_employed'] == 0), 'days_employed'] = median_days_5

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Отличный способ заполнить наши недостающие данные медианными показателями исходя из групп в данных.
    
В том числе можно было бы сделать функцию и применить её через метод `apply`.
    
</div>

Проверим, что нулевых значений не осталось:

In [49]:
data['days_employed'].value_counts()

1360     1106
1311      516
15020     414
2385      150
327        16
         ... 
15695       1
5466        1
3433        1
15727       1
2049        1
Name: days_employed, Length: 7789, dtype: int64

По стажу работы избавились от отрицательных значений, заполнили нулевые значения соответствующими медианными, преобразовали стаж для типа 'пенсионер'.

#### Обработка явных дубликатов

После преобразований всех столбцов найдём явные дубликаты в таблице методом `duplicated()` и посчитаем их количество методом `sum()`:

In [50]:
data.duplicated().sum()

71

В таблице есть 71 дубликат. Избавимся от них методом `drop_duplicates()` и сбросим индексы `reset_index()` c параметром `True`

In [51]:
data = data.drop_duplicates().reset_index(drop=True)

In [52]:
#Проверяем, что дубликатов не осталось
data.duplicated().sum()

0

In [53]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 12 columns):
children            21453 non-null int64
days_employed       21453 non-null int64
dob_years           21453 non-null int64
education           21453 non-null object
education_id        21453 non-null int64
family_status       21453 non-null object
family_status_id    21453 non-null int64
gender              21453 non-null object
income_type         21453 non-null object
debt                21453 non-null int64
total_income        21453 non-null int64
purpose             21453 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

На этапе обработки дубликатов проверили все столбцы на уникальные значения. Выявили артефакты и привели столбцы в презентабельный вид. 
Так же были найдены явные дубликаты. Возможно, дубликаты появились в ходе технического сбоя.

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Хорошо, что произвели текст в нижний регистр, как раз таким образом мы смогли устранить неявные дубликаты.
    
Сами дубликаты  нам ни к чему и очень здорово, что мы их удалили 😊
    
</div>

### Лемматизация

Лемматизируем столбец `'purpose'`, используя библиотеку для лемматизации:

In [54]:
#Напишем функцию, которая будет принимать на вход строку, разбивать текст в ячейке 'purpose' на леммы и возвращать их в ячейку 
def text_purpose (row):
    text = row [11]
    lemmas = m.lemmatize(text)
    row [11] = lemmas
    return row [11]

#Применим функцию к data, указав параметр axis =1 , чтобы передавались строки таблицы
data['purpose_lemmas'] = data.apply(text_purpose, axis = 1)
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]"
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]"
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]"
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]"
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,"[образование, \n]"
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]"
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]"


**Вывод**

После проведения лемматизации, по первым 15 строкам видно, что целями кредитов являются: недвижимость, жилье, автомобиль, образование, свадьба. Возможно, есть что-то ещё. В дальнейшем используем полученные данные для категоризации клиентов по целям

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Желательно конечно проверить наши данные по количеству, чтобы убедиться что категории мы точно выбрали верно. Например через метод `value_counts` 
    
</div>

### Категоризация данных

По таблице данных клиентов можно категоризировать:
1. По наличию детей
1. По семейному положению
1. По возрасту
1. По ежемесячному доходу
1. По целям на кредит

#### Категоризация по количеству детей у клиента

Разобьем клиентов на следующие категории:
1. Нет детей (`'children' == 0`)
1. Один ребенок (`'children' == 1`)
1. Два ребенка (`'children'== 2`)
1. Многодетный (`'children' >= 3`)

Напишем функцию, которой на вход будет попадать количество детей, а возвращать она будет категорию:

In [55]:
def child_category (children):
    if children == 0:
        return 'Нет детей'
    if children == 1:
        return 'Один ребенок'
    if children == 2:
        return 'Два ребенка'
    return 'Многодетный'

#добавляем столбец с категорией в таблицу
data['child_category'] = data['children'].apply(child_category)

data['child_category'].value_counts()

Нет детей       14090
Один ребенок     4855
Два ребенка      2128
Многодетный       380
Name: child_category, dtype: int64

После разделения по количеству детей оценим, какая доля кредитов приходится на каждую категорию:

In [56]:
part_child_category = data['child_category'].value_counts() / len(data) * 100
part_child_category

Нет детей       65.678460
Один ребенок    22.630867
Два ребенка      9.919359
Многодетный      1.771314
Name: child_category, dtype: float64

**Вывод**

Выполнили категоризацию клиентов банка по наличию детей. Более 60% кредитов выдали бездетным клиентам, почти пятая часть всех кредитов взята клиентами с одним ребенком. Около 10% взяли клиенты с двумя детьми. Кредиты многодетных клиентов составляют меньше двух процентов от всех кредитов.

#### Категоризация по семейному положению клиента

В таблице данных установим соответствие между `'family_status'` и `'family_status_id'`

In [57]:
#Создаем таблицу и переносим столбцы
family_dict = data[['family_status', 'family_status_id']]

#удаляем дубликаты из полученной таблицы
family_dict = family_dict.drop_duplicates().reset_index(drop = True)
family_dict

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


Таким образом, выявили следующее соответствие между семейным статусом и его id:
 - 0 - женат / замужем
 - 1 - гражданский брак
 - 2 - вдовец / вдова
 - 3 - в разводе
 - 4 - не женат / не замужем

Разделим всех клиентов на тех, кто состоит в отношениях ( статусы: 0, 1), и одиноких (статусы: 2, 3, 4). Напишем функцию, которая на вход получает `'family_status_id'`, а на выходе возвращать соответствующую категорию.

In [58]:
def family_category (family_status_id):
    if family_status_id == 0:
        return 'Состоит в отношениях'
    if family_status_id == 1:
        return 'Состоит в отношениях'
    if family_status_id == 2:
        return 'Не состоит в отношениях'
    if family_status_id == 3:
        return 'Не состоит в отношениях'
    return 'Не состоит в отношениях'

#добавляем столбец с категорией в таблицу
data['family_category'] = data['family_status_id'].apply(family_category)

data['family_category'].value_counts()

Состоит в отношениях       16489
Не состоит в отношениях     4964
Name: family_category, dtype: int64

После разделения по семейному положению оценим, какая доля кредитов приходится на каждую категорию:

In [59]:
part_family_category = data['family_category'].value_counts() / len(data) * 100
part_family_category

Состоит в отношениях       76.861045
Не состоит в отношениях    23.138955
Name: family_category, dtype: float64

При такой категоризации клиентов видно, что более 2/3 кредитов получили клиенты, состоящие в отношениях

#### Категоризация по возрасту клиента

Распределим клиентов по возрасту, по классификации Всемирной организации здравоохранения:
1. Молодой возраст - 18-44.
1. Средний возраст - 45-59.
1. Пожилой возраст - 60-74.
1. Старческий возраст - 75-90.

Напишем функцию, которая на вход получает `'dob_years'`, а на выходе возвращать соответствующую категорию.

In [60]:
def age_category (dob_years):
    if dob_years >= 18 and dob_years <= 44:
        return 'Молодой возраст'
    if dob_years >= 49 and dob_years <= 59:
        return 'Средний возраст'
    if dob_years >= 60 and dob_years <= 74:
        return 'Пожилой возраст'
    return 'Старческий возраст'

data['age_category'] = data['dob_years'].apply(age_category)
data['age_category'].value_counts()

Молодой возраст       11787
Средний возраст        5185
Пожилой возраст        2499
Старческий возраст     1982
Name: age_category, dtype: int64

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Отличная идея для новых категорий в наших данных 👍
    
</div>

Оценим долю взятых кредитов по категориям:

In [61]:
part_age_category = data['age_category'].value_counts() / len(data) * 100
part_age_category

Молодой возраст       54.943365
Средний возраст       24.169114
Пожилой возраст       11.648720
Старческий возраст     9.238801
Name: age_category, dtype: float64

Более половины кредитов выдано "Молодым" клиентам, чуть меньше четверти получили клиенты "Среднего" возраста. Доля "Пожилых" клиентов около 12%, доля "Стариков" - менее 10%

#### Категоризация по ежемесячным доходам

Можно попробовать разделить клиентов по ежемесячному доходу на категории:
1. Доход ниже среднего
1. Доход средний
1. Доход выше среднего

Для границы используем медианное значение зарплаты. К среднему доходу будем относить всех клиентов, чей доход отличается от медианного на не более, чем 25%. То есть 0,75* Median <= Средний доход <= 1.25* Median

In [62]:
#Находим медиану по доходу для всего столбца
median_income = data['total_income'].median().astype('int')
median_income

140212

In [63]:
# Напишем функцию , которая будет возвращать категорию в зависимости от дохода
def total_income_category (total_income):
    
    if total_income < 0.70 * median_income:
        return 'Доход ниже среднего'
    
    if total_income > 1.25 * median_income:
        return 'Доход выше среднего'
    
    return 'Доход средний'

data['total_income_category'] = data['total_income'].apply(total_income_category)
data['total_income_category'].value_counts()

Доход средний          10457
Доход выше среднего     6751
Доход ниже среднего     4245
Name: total_income_category, dtype: int64

Оценим долю взятых кредитов по категориям:

In [64]:
part_income_category = data['total_income_category'].value_counts() / len(data) * 100
part_income_category

Доход средний          48.743765
Доход выше среднего    31.468792
Доход ниже среднего    19.787442
Name: total_income_category, dtype: float64

Почти половину кредитов взято клиентами со средним доходом, 31, 5 % - кредиты клиентов с доходом выше среднего и 19,8% - с доходом ниже среднего

#### Категоризация по целям кредита

Разобьем цели кредита исходя из проведенной лемматизации на следующие категории:
1. Недвижимость
1. Жилье
1. Автомобиль
1. Образование
1. Свадьба
1. Другое

In [65]:
# Напишем функцию, которая принимает на вход список выделенных лемм, проверяет есть ли ключевое слово и возвращает категорию
def purpose_category (porpose_lemmas):
    
#    for purpose in porpose_lemmas:
        if 'недвижимость' in porpose_lemmas:
            return 'Недвижимость'
        
        if 'жилье' in porpose_lemmas:
            return 'Жилье'
        
        if 'автомобиль' in porpose_lemmas:
            return 'Автомобиль'
        
        if 'образование' in porpose_lemmas :
            return 'Образование'
        
        if 'свадьба' in porpose_lemmas:
            return 'Свадьба'
        
        return 'Другое'

data['purpose_category'] = data['purpose_lemmas'].apply(purpose_category)
data['purpose_category'].value_counts()

Недвижимость    6350
Жилье           4460
Автомобиль      4306
Образование     4013
Свадьба         2324
Name: purpose_category, dtype: int64

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Категории кредитов опредилили верно 👍
    
</div>

Оценим долю кредитов:

In [66]:
part_purpose_category = data['purpose_category'].value_counts() / len(data) * 100
part_purpose_category

Недвижимость    29.599590
Жилье           20.789633
Автомобиль      20.071785
Образование     18.706008
Свадьба         10.832984
Name: purpose_category, dtype: float64

Как видно после категоризации, выявились 5 конкретных категорий целей кредита. Категория "Другое" - отсутствует.
Больше всего клиенты берут кредит на недвижимость. Примерно в равной доле "Жилье", "Автомобиль" и "Образование".

#### Категоризация по полу клиента

In [67]:
data['gender'].value_counts()

F    14174
M     7279
Name: gender, dtype: int64

Оценим долю кредитов по полу:

In [68]:
data['gender'].value_counts() / len(data) * 100

F    66.070014
M    33.929986
Name: gender, dtype: float64

По таблице видим, что женщины берут кредит чаще. Их доля почти 2/3 всех кредитов

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Поищем взаимосвязь между возвратом кредита в срок и количеством детей. 

In [69]:
# сгруппируем данные по категории количества детей и найдем количество непогашенных кредитов методом agg(), суммировав по 'debt'
data_grouped_child = data.groupby(['child_category']).agg({'debt':sum})

# Найдем долю непогашенных кредитов для каждой категории поделив количество непогашенных кредитов на общее число кредитов категории в %
data_grouped_child['debt_part'] = data_grouped_child['debt'] / data['child_category'].value_counts() * 100

#Выведем общую таблицу с категорией, количеством непогашенных кредитов и их % от выданных кредитов на категорию
data_grouped_child.sort_values(by=['debt_part'])

,debt,debt_part
child_category,,
Нет детей,1063,7.544358
Многодетный,31,8.157895
Один ребенок,445,9.165808
Два ребенка,202,9.492481


**Вывод**

Определенно, прослеживается зависимость между возвратом кредита в срок и количеством детей. Клиенты без детей возвращают кредит лучше всех. С увеличением числа детей вероятность невозврата возрастает. Парадоксально, но по полученным данным, многодетные клиенты справляются с возвратом кредита лучше, чем клиенты с одним или двумя детьми. Вероятно, это связано с тем, что таких клиентов всего 1,77%. Возможно, стоит пересмотреть категоризацию, разделив на 3 группы: "Дети отсутствуют", "Один ребенок" и "Два и более детей".

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Верно 👍
    
Оставил пример ниже как можно было бы упроситить расчет.
    
</div>

In [70]:
data.groupby(['child_category']).agg({'debt':'mean'})

,debt
child_category,
Два ребенка,0.094925
Многодетный,0.081579
Нет детей,0.075444
Один ребенок,0.091658


- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [71]:
# сгруппируем данные по категории и найдем количество непогашенных кредитов методом agg(), суммировав по 'debt'
data_grouped_family = data.groupby(['family_category']).agg({'debt':sum})

# Найдем долю непогашенных кредитов для каждой категории поделив количество непогашенных кредитов на общее число кредитов категории в %
data_grouped_family['debt_part'] = data_grouped_family['debt'] / data['family_category'].value_counts() * 100

#Выведем общую таблицу с категорией, количеством непогашенных кредитов и их % от выданных кредитов на категорию
data_grouped_family.sort_values(by=['debt_part'])

,debt,debt_part
family_category,,
Состоит в отношениях,1319,7.999272
Не состоит в отношениях,422,8.501209


In [72]:
data_pivot_1 = data.pivot_table(index = 'family_category', columns = 'gender', values = 'debt', aggfunc='mean' )
data_pivot_1

gender,F,M
family_category,,
Не состоит в отношениях,0.064724,0.136918
Состоит в отношениях,0.071947,0.094494


**Вывод**

По таблице видим, что клиенты, состоящие в отношениях имеют много невыплаченных кредитов, при этом в целом возвращают их немного ( на 0,5%) лучше, чем одинокие клиенты

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

В точку 👍
    
Обрати внимание, что наши действия повторяются с точки зрения расчетов.
    
Это говорит о том, что мы можем создать функцию для постоянных расчетов. Тем самым мы сможем оптимизировать свой код 😊
    
</div>

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [73]:
# сгруппируем данные по категории и найдем количество непогашенных кредитов методом agg(), суммировав по 'debt'
data_grouped_income = data.groupby(['total_income_category']).agg({'debt':sum})

# Найдем долю непогашенных кредитов для каждой категории поделив количество непогашенных кредитов на общее число кредитов категории в %
data_grouped_income['debt_part'] = data_grouped_income['debt'] / data['total_income_category'].value_counts() * 100

#Выведем общую таблицу с категорией, количеством непогашенных кредитов и их % от выданных кредитов на категорию
data_grouped_income.sort_values(by=['debt_part'])

,debt,debt_part
total_income_category,,
Доход выше среднего,503,7.450748
Доход ниже среднего,342,8.056537
Доход средний,896,8.568423


In [74]:
data_pivot = data.pivot_table(index = 'total_income_category', values = ['debt'] , aggfunc='mean')
data_pivot

,debt
total_income_category,
Доход выше среднего,0.074507
Доход ниже среднего,0.080565
Доход средний,0.085684


**Вывод**

Зависимость не прослеживается. Клиенты с низким доходом выплачивают кредит лучше, чем клиенты со средним доходом, примерно на 0,5%  

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Верно 👍
    
В том числе можно было посчитать с помощью сводных таблиц `pivot_table`
    
</div>

- Как разные цели кредита влияют на его возврат в срок?

In [75]:
# сгруппируем данные по категории и найдем количество непогашенных кредитов методом agg(), суммировав по 'debt'
data_grouped_purpose = data.groupby(['purpose_category']).agg({'debt':sum})

# Найдем долю непогашенных кредитов для каждой категории поделив количество непогашенных кредитов на общее число кредитов категории в %
data_grouped_purpose['debt_part'] = data_grouped_purpose['debt'] / data['purpose_category'].value_counts() * 100

#Выведем общую таблицу с категорией, количеством непогашенных кредитов и их % от выданных кредитов на категорию
data_grouped_purpose.sort_values(by=['debt_part'])

,debt,debt_part
purpose_category,,
Жилье,308,6.905830
Недвижимость,474,7.464567
Свадьба,186,8.003442
Образование,370,9.220035
Автомобиль,403,9.359034


**Вывод**

Кредиты на Образование и Автомобиль возвращаются хуже всех, более 9%. Самая возвращаемая категория - на Жилье - менее 7%.

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

В точку 👍
    
В том числе можно было бы построить графики, например с помощью метода `hist` 
    
Как раз способы визуализации, мы уже будем проходить в следующем проекте 😊
    
</div>

- Посмотрим на возврат кредитов по возрастной категории:

In [76]:
# сгруппируем данные по категории и найдем количество непогашенных кредитов методом agg(), суммировав по 'debt'
data_grouped_years = data.groupby(['age_category']).agg({'debt':sum})

# Найдем долю непогашенных кредитов для каждой категории поделив количество непогашенных кредитов на общее число кредитов категории в %
data_grouped_years['debt_part'] = data_grouped_years['debt'] / data['age_category'].value_counts() * 100

#Выведем общую таблицу с категорией, количеством непогашенных кредитов и их % от выданных кредитов на категорию
data_grouped_years.sort_values(by=['debt_part'])

,debt,debt_part
age_category,,
Пожилой возраст,123,4.921969
Средний возраст,337,6.499518
Старческий возраст,149,7.517659
Молодой возраст,1132,9.603801


Хуже всех с выплатой справляются молодые (до 44) и старики (75-90). 

- Посмотрим на возврат кредитов по половой принадлежности:

In [77]:
# сгруппируем данные по категории и найдем количество непогашенных кредитов методом agg(), суммировав по 'debt'
data_grouped_gender = data.groupby(['gender']).agg({'debt':sum})

# Найдем долю непогашенных кредитов для каждой категории поделив количество непогашенных кредитов на общее число кредитов категории в %
data_grouped_gender['debt_part'] = data_grouped_gender['debt'] / data['gender'].value_counts() * 100

#Выведем общую таблицу с категорией, количеством непогашенных кредитов и их % от выданных кредитов на категорию
data_grouped_gender.sort_values(by=['debt_part'])

,debt,debt_part
gender,,
F,994,7.012840
M,747,10.262399


Женщины оказались более ответственными. Больше кредитов берут и возвращают лучше мужчин.

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Отличные инсайты мы собрали для наших коллег 👍
    
</div>

## Шаг 4. Общий вывод

### Предобработка данных

Провели замену пропусков, удаление дубликатов, проверили таблицу на корректность ифнормации.

### Категоризация клиентов

Провели категоризацию клиентов:
1. По количеству детей
1. По семейному положению
1. По возрасту клиента
1. По ежемесячному доходу
1. По полу клиента
1. По целям кредита

### Главные выводы и расчеты

Расчитали процентное соотношение непогашенных кредитов к общему числу взятых кредитов по выявленным категориям клиентов

1. Установили, что с увеличением количества детей, вероятность невозврата кредита увеличивается.
1. Установили, что люди, состоящие в отношениях (гражданский брак, женат/замужем) выплачивают кредит лучше, чем одинокие.
1. Не установили прямой зависимости между уровнем дохода и погашением кредита. Клиенты с низким доходом выплачивают лучше, чем клиенты со средним доходов.
1. Не установили прямой зависимости от возраста. Молодые и старики возвращают кредит хуже.
1. Установили, что хуже всего клиенты возвращают кредиты, взятые на "Автомобиль" и "Образование".
1. Установили, что клиенты женского пола выплачивают кредит лучше мужского.


### Рекомендации

Возможно стоит на этапе сбора информации предусмотреть обязательность ввода некоторых полей: "Стаж", "Доход", "Пол". Для полей "Количество детей", "Стаж" стоит предусмотреть валидацию или проверку на неотрицательные значения.

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Выводы понятны и логичны, а самое главное подкреплены выявленными фактами.
       
В целом сам проект выполнен на хорошем уровне! На протяжении всей работы чувствуется глубина проработки задачи и это безусловный плюс для нас.
    
Хочется отметить, что мы узнали достаточно много нового в этом проекте и с уверенностью можем двигаться дальше. На что рекомендую обращать внимание сразу с самого старта так сказать, а именно на повторяемость кода. Если наши действия начинают повторяться более 2-х раз, то имеет смысл задуматься над: 
    
- внедрением цикла 
- внедрением функции
- внедрением комбинации цикл + функция
    
Такие вещи могут сразу не бросаться в глаза, но постепенно мы придём к такому подходу 😊
    
Поздравляю с успешным завершением проекта 😊👍
    
И желаю успехов в новых работах 😊
    
**[общий комментарий]** Не стоит забывать про оформление своей работы, это важный такой же важный этап. Рекомендую «не стесняться» использовать ячейки типа **markdown** там где нам необходимо отобразить свои размышления, в том числе с использованием дополнительной стилизации.
    
https://sqlbak.com/blog/jupyter-notebook-markdown-cheatsheet
    
    
</div>

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.